# **Exercício Prático - Aula 17:**

### **Nome: Isabella Maria Ribeiro - RA: 168870.**

In [13]:
import random
from collections import deque
import numpy as np

# MATRIZ: 1-Wumpus; 2- cheiro ruim; 3- abismo; 4- brisa; 5- ouro; 6- cheiro ruim e brisa; 7- cheiro ruim e ouro; 8- brisa e ouro; 9- cheiro ruim, ouro, e brisa.

matriz = [[0,0,0,0],
          [0,0,0,0],
          [0,0,0,0],
          [0,0,0,0]]

def poscheiro(ccrx,ccry):
  if (ccrx <= 3 and ccrx >= 0) and (ccry<=3 and ccry >=0):
    matriz[ccrx][ccry] = 2

def posbrisa(bbrx,bbry):
  if (bbrx <= 3 and bbrx >= 0) and (bbry<=3 and bbry >=0):
    if matriz[bbrx][bbry] == 0:
        matriz[bbrx][bbry] = 4
    if matriz[bbrx][bbry] == 2:
      matriz[bbrx][bbry] = 6

def ambiente():
  for i in range(4):
    for j in range(4):
      matriz[i][j] = 0

  #Wumpus:
  wx = random.randint(1,3)
  wy = random.randint(1,3)
  matriz[wx][wy] = 1

  #Colocando cheiro ruim:
  crx = wx+1
  cry = wy
  poscheiro(crx,cry)

  crx = wx-1
  cry = wy
  poscheiro(crx,cry)

  crx = wx
  cry = wy+1
  poscheiro(crx,cry)

  crx = wx
  cry = wy-1
  poscheiro(crx,cry)

  #Colocando abismo:
  for i in range(3):
    abx = random.randint(1,3)
    aby = random.randint(1,3)
    while matriz[abx][aby] != 0:
      abx = random.randint(1,3)
      aby = random.randint(1,3)
    matriz[abx][aby] = 3

    brx = abx+1
    bry = aby
    posbrisa(brx,bry)

    brx = abx-1
    bry = aby
    posbrisa(brx,bry)

    brx = abx
    bry = aby+1
    posbrisa(brx,bry)

    brx = abx
    bry = aby-1
    posbrisa(brx,bry)


  #Colocando Ouro:
  ox = random.randint(1,3)
  oy = random.randint(1,3)
  while matriz[ox][oy] ==1 or matriz[ox][oy] ==3:
    ox = random.randint(1,3)
    oy = random.randint(1,3)
  if matriz[ox][oy] == 0:
    matriz[ox][oy] = 5
  elif(matriz[ox][oy] ==2):
    matriz[ox][oy] = 7
  elif(matriz[ox][oy] ==4):
    matriz[ox][oy] = 8
  elif(matriz[ox][oy] ==6):
    matriz[ox][oy] = 9

  return matriz

def menuinformativo():
  print("*************************")
  print("      Jogo do Wumpus     ")
  print("Objetivo: Pegar o ouro e voltar à entrada, evitando os perigos.")
  print("Pontuação: ganha 50 pontos por matar o Wumpus, 50 pontos por escalar a caverna para fora carregando o ouro com 1 ponto a menos para cada ação e 100 pontos negativos por ser morto.")
  print("*************************")

def menu1():
  print("Menu Principal")
  print("1 - Jogar sozinho")
  print("2 - Agente Inteligente (Usando Busca)")
  print("3 - Agente Inteligente (Usando Q-Learning)")
  print("4 - Sair")

def menu2():
  print("Menu Interno")
  print("1 - Mover para Cima")
  print("2 - Mover para Baixo")
  print("3 - Mover para Direita")
  print("4 - Mover para Esquerda")
  print("5 - Posicionar Flecha")

def bfs_caminho(origem, destino, seguras):
    if origem == destino:
        return []

    visitados = set([origem])
    fila = deque([(origem, [])])  # (posição, caminho)

    while fila:
        pos, caminho = fila.popleft()

        # Verificar células adjacentes
        for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
            nx, ny = pos[0] + dx, pos[1] + dy
            prox = (nx, ny)

            if 0 <= nx < 4 and 0 <= ny < 4 and prox not in visitados:
                if prox == destino or prox in seguras:
                    novo_caminho = caminho + [prox]
                    if prox == destino:
                        return novo_caminho

                    visitados.add(prox)
                    fila.append((prox, novo_caminho))

    return []  # Não encontrou caminho


def agente_bfs_if():
    ambiente()

    # ambiente gerado
    #for i in range(3, -1, -1):
        #print(matriz[i])
    #print()

    # Inicialização de variáveis
    jogadorx, jogadory = 0, 0
    visitadas = set([(jogadorx, jogadory)])
    fila = deque([(jogadorx, jogadory)])
    seguras = set([(jogadorx, jogadory)])
    suspeitas = set()  # Células perigosas
    caminho = {}  # caminho percorrido
    flecha = 1
    ouro = 0
    wumpus = 0
    pontos = 0

    print("****AGENTE INTELIGENTE está jogando****")

    # Loop principal
    while True:
        # Avaliar a célula atual
        print(f"\nExplorando agente está na posição ({jogadorx+1}, {jogadory+1})")
        celula_atual = matriz[jogadorx][jogadory]

        # Verificar se morreu
        if celula_atual == 1:
            print("O agente encontrou o Wumpus e MORREU :( ")
            pontos = -100
            print(f"Pontos finais: {pontos}")
            print("******************************")
            print("********** FIM DE JOGO! **********")
            print("******************************")
            return
        elif celula_atual == 3:
            print("O agente caiu em um abismo e MORREU")
            pontos = -100
            print(f"Pontos finais: {pontos}")
            print("******************************")
            print("********** FIM DE JOGO! **********")
            print("******************************")
            return

        # Percepções
        if celula_atual == 2:
            print("Agente percebe fedor nesta posição")
            # Células adjacentes pode ter Wumpus
            for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
                nx, ny = jogadorx + dx, jogadory + dy
                if 0 <= nx < 4 and 0 <= ny < 4:
                    if (nx, ny) not in visitadas and (nx, ny) not in seguras:
                        suspeitas.add((nx, ny))
        elif celula_atual == 4:
            print("Agente percebe brisa nesta posição")
            # Células adjacentes pode ter abismo
            for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
                nx, ny = jogadorx + dx, jogadory + dy
                if 0 <= nx < 4 and 0 <= ny < 4:
                    if (nx, ny) not in visitadas and (nx, ny) not in seguras:
                        suspeitas.add((nx, ny))
        elif celula_atual == 5:
            print("Agente percebe um brilho! OURO ENCONTRADO!")
            ouro = 1
            pontos += 50
            matriz[jogadorx][jogadory] = 0
        elif celula_atual == 6:
            print("Agente percebe fedor e brisa nesta posição")
            # Células adjacentes podem ter Wumpus ou abismo
            for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
                nx, ny = jogadorx + dx, jogadory + dy
                if 0 <= nx < 4 and 0 <= ny < 4:
                    if (nx, ny) not in visitadas and (nx, ny) not in seguras:
                        suspeitas.add((nx, ny))
        elif celula_atual == 7:
            print("Agente percebe fedor e brilho! OURO ENCONTRADO!")
            ouro = 1
            pontos += 50
            matriz[jogadorx][jogadory] = 2  # Removendo o ouro porém deixando o fedor
            # Células adjacentes pode ter Wumpus
            for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
                nx, ny = jogadorx + dx, jogadory + dy
                if 0 <= nx < 4 and 0 <= ny < 4:
                    if (nx, ny) not in visitadas and (nx, ny) not in seguras:
                        suspeitas.add((nx, ny))
        elif celula_atual == 8:
            print("Agente percebe brisa e brilho! OURO ENCONTRADO!")
            ouro = 1
            pontos += 50
            matriz[jogadorx][jogadory] = 4  # Removendo o ouro porém deixando a brisa
            #  Células adjacentes pode ter abismo
            for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
                nx, ny = jogadorx + dx, jogadory + dy
                if 0 <= nx < 4 and 0 <= ny < 4:
                    if (nx, ny) not in visitadas and (nx, ny) not in seguras:
                        suspeitas.add((nx, ny))
        elif celula_atual == 9:
            print("Agente percebe fedor, brisa e brilho! OURO ENCONTRADO!")
            ouro = 1
            pontos += 50
            matriz[jogadorx][jogadory] = 6  # Removendo o ouro porém mantendo fedor e brisa
            # Células adjacentes podem ter abismo ou wumpus
            for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
                nx, ny = jogadorx + dx, jogadory + dy
                if 0 <= nx < 4 and 0 <= ny < 4:
                    if (nx, ny) not in visitadas and (nx, ny) not in seguras:
                        suspeitas.add((nx, ny))
        else:
            print("Agente NÃO TEM NENHUMA percepção nesta posição")
            # Células adjacentes são seguras
            for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
                nx, ny = jogadorx + dx, jogadory + dy
                if 0 <= nx < 4 and 0 <= ny < 4:
                    seguras.add((nx, ny))
                    if (nx, ny) in suspeitas:
                        suspeitas.remove((nx, ny))

        if ouro == 1 and jogadorx == 0 and jogadory == 0:
            print("VITÓRIA! O agente conseguiu retornar até a entrada com o ouro!")
            pontos += 50
            print("Pontos finais: ", pontos)
            print("******************************")
            print("********** FIM DE JOGO! **********")
            print("******************************")
            return
        elif ouro == 1:
            print("OBJETIVO CONQUISTADO: Ouro encontrado! Voltando para a entrada (1,1)")
            # Usando Busca em Largura para encontrar caminho de volta
            caminho_volta = bfs_caminho((jogadorx, jogadory), (0, 0), seguras)

            if not caminho_volta:
                print("Há um problema, não foi possível encontrar caminho seguro de volta à entrada!")
                print("Movimentação para uma posição ARRISCADA")

                melhor_movimento = None
                menor_distancia = float('inf')

                for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
                    nx, ny = jogadorx + dx, jogadory + dy
                    if 0 <= nx < 4 and 0 <= ny < 4:
                        distancia = nx + ny  # Distância de Manhattan
                        if distancia < menor_distancia:
                            menor_distancia = distancia
                            melhor_movimento = (nx, ny)

                if melhor_movimento:
                    print(f"Arriscando: Movendo para ({melhor_movimento[0]+1}, {melhor_movimento[1]+1})")
                    jogadorx, jogadory = melhor_movimento
                    pontos -= 1
                else:
                    print("Não há posição adjacente disponível!")
                    print(f"Pontuação final: {pontos}")
                    return
                continue

            # Caminho de volta
            prox_x, prox_y = caminho_volta[0]
            print(f"Caminho de Volta: Movendo para ({prox_x+1}, {prox_y+1})")
            jogadorx, jogadory = prox_x, prox_y
            pontos -= 1  #
            continue

        # Tentar usar a flecha se detectar fedor
        if flecha > 0 and (celula_atual == 2 or celula_atual == 6 or celula_atual == 7 or celula_atual == 9) and wumpus == 0:
            # Achando Wumpus
            possivel_wumpus = []
            for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
                nx, ny = jogadorx + dx, jogadory + dy
                if 0 <= nx < 4 and 0 <= ny < 4:
                    if (nx, ny) in suspeitas and (nx, ny) not in visitadas:
                        possivel_wumpus.append((nx, ny))

            if possivel_wumpus:
                wx, wy = possivel_wumpus[0]  # Escolhe uma das posições
                print(f"Suspeitando de WUMPUS por perto! Atirando flecha para a posição ({wx+1}, {wy+1})")
                flecha -= 1

                if matriz[wx][wy] == 1:
                    print("WUMPUS MORREU!")
                    matriz[wx][wy] = 0
                    wumpus = 1
                    pontos += 50

                    # Remover fedores
                    for i in range(4):
                        for j in range(4):
                            if matriz[i][j] == 2:
                                matriz[i][j] = 0
                            elif matriz[i][j] == 6:
                                matriz[i][j] = 4
                            elif matriz[i][j] == 7:
                                matriz[i][j] = 5
                            elif matriz[i][j] == 9:
                                matriz[i][j] = 8

                    seguras.add((wx, wy))
                    if (wx, wy) in suspeitas:
                        suspeitas.remove((wx, wy))
                else:
                    print("A flecha não acertou o WUMPUS - Agente não possui mais flecha")

        visitadas.add((jogadorx, jogadory))

        # Fila
        if fila and fila[0] == (jogadorx, jogadory):
            fila.popleft()

        # Colocando células adjacentes seguras não visitadas na fila
        for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
            nx, ny = jogadorx + dx, jogadory + dy
            if 0 <= nx < 4 and 0 <= ny < 4:
                if (nx, ny) in seguras and (nx, ny) not in visitadas and (nx, ny) not in fila:
                    fila.append((nx, ny))
                    caminho[(nx, ny)] = (jogadorx, jogadory)

        # Sem posições seguras
        if not fila:
            print("Não há posições seguras para explorar.")

            if ouro == 0:
                print("O ouro ainda não foi encontrado. Agente irá tentar arriscar...")

            # Tentar mover para qualquer posição adjacente não visitada
            movido = False
            for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
                nx, ny = jogadorx + dx, jogadory + dy
                if 0 <= nx < 4 and 0 <= ny < 4 and (nx, ny) not in visitadas:
                    print(f"Arriscando: Movendo para ({nx+1}, {ny+1})")
                    jogadorx, jogadory = nx, ny
                    pontos -= 1
                    movido = True
                    break

            if not movido:
                print("Não há posição adjacente disponível para explorar!")
                print("Tentando retornar para a entrada")

                # Tentar voltar para a entrada mesmo sem caminho seguro
                if jogadorx > 0:
                    print(f"Arriscando: Movendo para ({jogadorx}, {jogadory+1})")
                    jogadorx -= 1
                    pontos -= 1
                elif jogadory > 0:
                    print(f"Arriscando: Movendo para ({jogadorx+1}, {jogadory})")
                    jogadory -= 1
                    pontos -= 1
                else:
                    print("Agente a está na entrada e não pode fazer mais nada")
                    print(f"Pontos finais: {pontos}")
                    return
            continue

        # Mover para a próxima da fila
        if fila:
            proximo = fila[0]

            if abs(proximo[0] - jogadorx) + abs(proximo[1] - jogadory) == 1:
                prox_x, prox_y = proximo
                print(f"Agente se movendo para ({prox_x+1}, {prox_y+1})")
                jogadorx, jogadory = prox_x, prox_y
                pontos -= 1
            else:
                caminho_para_proximo = bfs_caminho((jogadorx, jogadory), proximo, seguras)

                if caminho_para_proximo:
                    prox_x, prox_y = caminho_para_proximo[0]
                    print(f"Agente se movendo para ({prox_x+1}, {prox_y+1})")
                    jogadorx, jogadory = prox_x, prox_y
                    pontos -= 1
                else:
                    # Se arriscando
                    print("Não há posições seguras para explorar.")
                    print("Agente irá tentar arriscar...")

                    movido = False
                    for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
                        nx, ny = jogadorx + dx, jogadory + dy
                        if 0 <= nx < 4 and 0 <= ny < 4 and (nx, ny) not in visitadas:
                            print(f"Arriscando: Movendo para ({nx+1}, {ny+1})")
                            jogadorx, jogadory = nx, ny
                            pontos -= 1
                            movido = True
                            break

                    if not movido:
                        # Não conseguiu se movimentar
                        fila.popleft()
                        print("Não foi possível mover para nenhuma posição adjacente")
                        if not fila:
                            print("Não há mais células para explorar com segurança")
                            print(f"Pontos finais: {pontos}")
                            return

def agente_q_learning():
    print("****AGENTE INTELIGENTE (Q-LEARNING) está treinando****")

    #Parametros
    num_ep = 100
    max_ep = 10

    taprend = 0.1  #Alpha
    fatordesc = 0.99   #Gamma

    texplo = 1.0   #ação aleatória
    maxexplo = 1.0
    minexplo = 0.01
    taxadecai = 0.001

    #Inicialização da Tabela Q
    Q = np.zeros((4, 4, 4)) #Formato:[linha][coluna][ação]

    #Recompensas
    recom_ep = []

    ambiente()
    print("Ambiente já gerado!!")

    #Treinamento:
    for episodio in range(num_ep):
        jogadorx, jogadory = 0, 0
        rtotal = 0
        terminou = False

        for passo in range(max_ep):
            estado_atual = (jogadorx, jogadory)

            if random.uniform(0, 1) > texplo:
                acao = np.argmax(Q[jogadorx, jogadory, :])
            else:
                #ação aleatória
                acao = random.randint(0, 3)

            proxx, proxy = jogadorx, jogadory
            if acao == 0: #Cima
                proxx = min(jogadorx + 1, 3)
            elif acao == 1: #Baixo
                proxx = max(jogadorx - 1, 0)
            elif acao == 2: #Direita
                proxy = min(jogadory + 1, 3)
            elif acao == 3: #Esquerda
                proxy = max(jogadory - 1, 0)

            recompensa = 0
            celula_destino = matriz[proxx][proxy]

            #Checa os valores na matriz de ambiente
            if celula_destino in [1, 3]: #Wumpus ou Abismo
                recompensa = -1000
                terminou = True
            elif celula_destino in [5, 7, 8, 9]: #Ouro
                recompensa = 1000
                terminou = True
            else: #Movimento normal
                recompensa = -1 #Perde por passo

            #Atualizando Q
            estado_antigo = estado_atual
            valor_antigo= Q[estado_antigo[0], estado_antigo[1], acao]

            proximo_max = np.max(Q[proxx, proxy, :])

            novo_q = valor_antigo + taprend * (recompensa + fatordesc * proximo_max - valor_antigo)
            Q[estado_antigo[0], estado_antigo[1], acao] = novo_q

            jogadorx, jogadory = proxx, proxy
            rtotal += recompensa

            if terminou:
                break

        # --- Decaimento da Taxa de Exploração ---
        texplo = minexplo + \
            (maxexplo - minexplo) * np.exp(-taxadecai * episodio)

        recom_ep.append(rtotal)

    print("Treinamento terminado!\n")
    print("Tabela Q final:\n", Q)

    print("****Executando o agente treinado****")

    jogadorx, jogadory = 0, 0
    pontos = 0
    passos_finais = 0

    for passo in range(max_ep):
        print(f"Passo {passo+1}: Agente está em ({jogadorx+1}, {jogadory+1})")

        # Escolhe a melhor ação da tabela Q
        acao = np.argmax(Q[jogadorx, jogadory, :])
        acoes_str = ["Cima", "Baixo", "Direita", "Esquerda"]
        print(f"Ação escolhida: {acoes_str[acao]}")

        # Move o agente
        if acao == 0:
          jogadorx = min(jogadorx + 1, 3)
        elif acao == 1:
          jogadorx = max(jogadorx - 1, 0)
        elif acao == 2:
          jogadory = min(jogadory + 1, 3)
        elif acao == 3:
          jogadory = max(jogadory - 1, 0)

        pontos -= 1 #Custo a cada movimento
        passos_finais +=1

        # Verifica o resultado
        celula_final = matriz[jogadorx][jogadory]
        if celula_final in [1, 3]:
            print(f"O agente entrou em ({jogadorx+1}, {jogadory+1}) e MORREU! (Wumpus ou Poço)")
            pontos -= 1000
            break
        elif celula_final in [5, 7, 8, 9]:
            print(f"O agente entrou em ({jogadorx+1}, {jogadory+1}) e ACHOU O OURO!")
            pontos += 1000
            break
        elif passos_finais >= max_ep-1:
            print("Agente atingiu o limite de passos. Falhou em encontrar o ouro.")
            break

    print("\n******************************")
    print("********** FIM DE JOGO! **********")
    print(f"Pontuação final do agente Q-Learning: {pontos}")
    print("******************************")

menuinformativo()
print()
menu1()
op = int(input("Escolha uma opção: "))
print()
while(op != 1 and op != 2 and op!=3 and op!=4):
  print("Opção inválida!")
  op = int(input("Escolha outra opção: "))
  print()

while op != 4:
  while op == 1:
    ambiente()
    #Só para ver se estava funcionando corretamente:
    #i=3
    #while i>=0:
    #  print(matriz[i])
    #  i-=1

    print("Opção 'Jogar Sozinho':")
    print("*************************")
    print("    Bem vindo ao jogo!   ")
    print("*************************")
    print()
    pontos=0
    jogadorx = 0
    jogadory = 0
    flecha = 1
    ouro = 0
    wumpus=0

    while pontos!=-100:
      print("*************************")
      print("Pontos = ", pontos)
      print("Flechas = ", flecha)
      print()

      if wumpus ==1:
        for i in range(4):
          for j in range(4):
            if matriz[i][j] == 2:
              matriz[i][j] = 0
            elif matriz[i][j] == 6:
              matriz[i][j] = 4
            elif matriz[i][j] == 7:
              matriz[i][j] = 5
            elif matriz[i][j] == 9:
              matriz[i][j] = 8


      if ouro == 1 and wumpus==1:
        pontos-=1
      if matriz[jogadorx][jogadory] == 1:
        print("WUMPUS NESSA POSIÇÃO! VOCÊ MORREU!")
        pontos = -100
        print("******************************")
        print("********** FIM DE JOGO! **********")
        print("******************************")
        break
      elif matriz[jogadorx][jogadory] == 2:
        print(f"Você está na posição [{jogadorx+1},{jogadory+1}] e percebe um fedor.")
      elif matriz[jogadorx][jogadory] == 3:
        print("ABISMO NESSA POSIÇÃO! VOCÊ MORREU!")
        print("******************************")
        print("********** FIM DE JOGO! **********")
        print("******************************")
        pontos = -100
        break
      elif matriz[jogadorx][jogadory] == 4 :
        print(f"Você está na posição [{jogadorx+1},{jogadory+1}] e percebe uma brisa.")
      elif matriz[jogadorx][jogadory] == 5 :
        print(f"Você está na posição [{jogadorx+1},{jogadory+1}] e percebe um brilho.")
        print("Você achou o ouro e recebeu 50 pontos!")
        ouro = 1
        pontos+=50
        matriz[jogadorx][jogadory] = 0
      elif matriz[jogadorx][jogadory] == 6 :
        print(f"Você está na posição [{jogadorx+1},{jogadory+1}] e percebe um fedor e uma brisa.")
      elif matriz[jogadorx][jogadory] == 7 :
        print(f"Você está na posição [{jogadorx+1},{jogadory+1}] e percebe um fedor e um brilho.")
        print("Você achou o ouro e recebeu 50 pontos!")
        ouro = 1
        pontos+=50
        matriz[jogadorx][jogadory] = 2
      elif matriz[jogadorx][jogadory] == 8 :
        print(f"Você está na posição [{jogadorx+1},{jogadory+1}] e percebe uma brisa e um brilho.")
        print("Você achou o ouro e recebeu 50 pontos!")
        ouro = 1
        pontos+=50
        matriz[jogadorx][jogadory] = 4
      elif matriz[jogadorx][jogadory] == 9 :
        print(f"Você está na posição [{jogadorx+1},{jogadory+1}] e percebe um fedor, uma brisa e um brilho.")
        print("Você achou o ouro e recebeu 50 pontos!")
        ouro = 1
        pontos+=50
        matriz[jogadorx][jogadory] = 6

      elif jogadorx==0 and jogadory==0 and ouro==1 and wumpus==1:
        print(f"PARABÉNS! VOCÊ GANHOU O JOGO.")
        print("Pontos finais: ", pontos)
        print("******************************")
        print("********** FIM DE JOGO! **********")
        print("******************************")
        break
      else:
        print(f"Você está na posição [{jogadorx+1},{jogadory+1}] e não percebe nada.")
      print()

      menu2()
      op2 = int(input("Escolha uma opção: "))
      while(op2>5 or op2<1):
        print("Opção inválida!")
        op2 = int(input("Escolha outra opção: "))
        print()
      print()

      #Movendo para cima
      if op2 == 1:
        if (jogadorx+1<=3 and jogadorx+1>=0) and (jogadory<=3 and jogadory>=0):
          jogadorx+=1
        else:
          print("Você levou um choque por tentar ultrapassar uma parede!")
          print()

      #Movendo para Baixo
      elif op2 == 2:
        if (jogadorx-1<=3 and jogadorx-1>=0) and (jogadory<=3 and jogadory>=0):
          jogadorx-=1
        else:
          print("Você levou um choque por tentar ultrapassar uma parede!")
          print()

      #Movendo para direita
      elif op2 == 3:
        if (jogadorx<=3 and jogadorx>=0) and (jogadory+1<=3 and jogadory+1>=0):
          jogadory+=1
        else:
          print("Você levou um choque por tentar ultrapassar uma parede!")
          print()

      #Movendo para esquerda
      elif op2 == 4:
        if (jogadorx<=3 and jogadorx>=0) and (jogadory-1<=3 and jogadory-1>=0):
          jogadory-=1
        else:
          print("Você levou um choque por tentar ultrapassar uma parede!")
          print()

      else:
        if flecha == 0:
          print("Você não tem mais flechas!")
        else:
          flecha-=1
          print("Você deseja jogar a flecha para onde?")
          print("1 - Cima")
          print("2 - Baixo")
          print("3 - Direita")
          print("4 - Esquerda")
          op3 = int(input("Escolha uma opção: "))
          while(op3>4 or op3<1):
            print("Opção inválida!")
            op3 = int(input("Escolha outra opção: "))
            print()
          if op3 == 1:
            if (jogadorx+1<=3 and jogadorx+1>=0) and (jogadory<=3 and jogadory>=0):
              if matriz[jogadorx+1][jogadory] == 1:
                print()
                print("WUMPUS NESSA POSIÇÃO! VOCÊ o MATOU!")
                print("O Wumpus é morto e solta um grito. Você recebeu 50 pontos!")
                pontos +=50
                matriz[jogadorx+1][jogadory] = 0
                wumpus=1
                print("Pontos: ", pontos)
              else:
                print("Você errou a flecha!")
            else:
              print("Você errou a flecha!")

          elif op3 == 2:
            if (jogadorx-1<=3 and jogadorx-1>=0) and (jogadory<=3 and jogadory>=0):
              if matriz[jogadorx-1][jogadory] == 1:
                print()
                print("WUMPUS NESSA POSIÇÃO! VOCÊ o MATOU!")
                print("O Wumpus é morto e solta um grito. Você recebeu 50 pontos!")
                pontos +=50
                print("Pontos: ", pontos)
                matriz[jogadorx-1][jogadory] = 0
                wumpus=1
              else:
                print("Você errou a flecha!")
            else:
              print("Você errou a flecha!")

          elif op3 == 3:
            if (jogadorx<=3 and jogadorx>=0) and (jogadory+1<=3 and jogadory+1>=0):
              if matriz[jogadorx][jogadory+1] == 1:
                print()
                print("WUMPUS NESSA POSIÇÃO! VOCÊ o MATOU!")
                print("O Wumpus é morto e solta um grito. Você recebeu 50 pontos!")
                pontos +=50
                matriz[jogadorx][jogadory+1] = 0
                wumpus=1
                print("Pontos: ", pontos)
              else:
                print("Você errou a flecha!")
            else:
              print("Você errou a flecha!")

          elif op3 == 4:
            if (jogadorx<=3 and jogadorx>=0) and (jogadory-1<=3 and jogadory-1>=0):
              if matriz[jogadorx][jogadory-1] == 1:
                print()
                print("WUMPUS NESSA POSIÇÃO! VOCÊ o MATOU!")
                print("O Wumpus é morto e solta um grito. Você recebeu 50 pontos!")
                pontos +=50
                matriz[jogadorx][jogadory-1] = 0
                wumpus=1
              else:
                print("Você errou a flecha!")
            else:
              print("Você errou a flecha!")
    print()
    menu1()
    op = int(input("Escolha uma opção: "))
    while(op != 1 and op != 2 and op !=3 and op!=4):
      print("Opção inválida!")
      op = int(input("Escolha outra opção: "))
      print()
    print()

  while op == 2:
    print()
    print("*************************")
    print("    Bem vindo ao jogo!   ")
    print("*************************")
    print()
    agente_bfs_if()
    print()
    menu1()
    op = int(input("Escolha uma opção: "))
    while(op != 1 and op != 2 and op !=3 and op!=4):
      print("Opção inválida!")
      op = int(input("Escolha outra opção: "))
      print()

  while op == 3:
    print()
    print("*************************")
    print("    Bem vindo ao jogo!   ")
    print("*************************")
    print()
    agente_q_learning()
    print()
    menu1()
    op = int(input("Escolha uma opção: "))
    while(op != 1 and op != 2 and op !=3 and op!=4):
      print("Opção inválida!")
      op = int(input("Escolha outra opção: "))
      print()

print()
print("**************************************")
print("Obrigada por se divertir aqui! Até a próxima!")
print("**************************************")
print()

*************************
      Jogo do Wumpus     
Objetivo: Pegar o ouro e voltar à entrada, evitando os perigos.
Pontuação: ganha 50 pontos por matar o Wumpus, 50 pontos por escalar a caverna para fora carregando o ouro com 1 ponto a menos para cada ação e 100 pontos negativos por ser morto.
*************************

Menu Principal
1 - Jogar sozinho
2 - Agente Inteligente (Usando Busca)
3 - Agente Inteligente (Usando Q-Learning)
4 - Sair
Escolha uma opção: 3


*************************
    Bem vindo ao jogo!   
*************************

****AGENTE INTELIGENTE (Q-LEARNING) está treinando****
Ambiente já gerado!!
Treinamento terminado!

Tabela Q final:
 [[[ 9.41566326e+02  8.89688484e+02  8.40206421e+02  8.22551385e+02]
  [ 9.01522910e+02  4.13581927e+02  1.70896371e+02  6.94160834e+02]
  [-4.09510000e+02  5.62050491e+01  5.69297780e-02  3.46394792e+02]
  [-1.00000000e-01 -1.00000000e-01 -1.90000000e-01  3.57621264e+01]]

 [[ 3.84434612e+02  6.38609645e+02  9.79724440e+02  7.9666214

# **Fontes Consultadas:**
Aula 17 - Aprendizado por reforço - Lilian Berton.

https://gemini.google.com/
